## K-Fold Cross Validation with Hyper parameter tuning for 7 ML Models
- K = 5  
- tuner: keras-tuner / hyperopt  
- ML Models:
    * RF (Random Forest)
    * XGBoost
    * LightGBM
    * LR (Linear Regression)
    * SVR (Support Vector Regression)
    * DNN (Deep Neural Network)
    * AE (Auto Encoder)

---

### 0-1. Library Import

In [13]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_val_predict
from sklearn.metrics import mean_squared_error, root_mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, make_scorer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb

from hyperopt import hp, tpe, Trials, fmin, STATUS_OK
from hyperopt.pyll.base import scope

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.losses import MeanAbsoluteError
from tensorflow.python.client import device_lib

import keras_tuner as kt
from kerastuner.tuners import RandomSearch, Hyperband
from kerastuner.engine.hyperparameters import HyperParameters

from joblib import Parallel, delayed

In [14]:
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

In [15]:
# print model test results
def print_test_results(y_t, pred):
    print(f'[Test results]\n'
          f'MAPE: {round(mean_absolute_percentage_error(y_t, pred), 2)}\n'
          f'MAE: {round(mean_absolute_error(y_t, pred), 2)}\n'
          f'MSE: {round(mean_squared_error(y_t, pred), 2)}\n'
          f'RMSE: {round(root_mean_squared_error(y_t, pred), 2)}\n')

# Return model test results
def get_regression_metrics(y_t, pred):
    MAPE = round(mean_absolute_percentage_error(y_t, pred), 2)
    MAE = round(mean_absolute_error(y_t, pred), 2)
    MSE = round(mean_squared_error(y_t, pred), 2)
    RMSE = round(root_mean_squared_error(y_t, pred), 2)
    
    return MAPE, MAE, MSE, RMSE

### 0-2. Get GPU

In [16]:
# GPU Test
print(tf.__version__)
print("==========================================")
print(tf.test.is_gpu_available())
print("==========================================")
print(tf.test.is_built_with_cuda())
print("==========================================")
print(tf.test.gpu_device_name())
print("==========================================")
device_lib.list_local_devices()

2.10.0
True
True
/device:GPU:0


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 11051245953005723391
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 10057940992
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 4410671303411310337
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3060, pci bus id: 0000:01:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

### 0-3. Data import

In [17]:
raw_df = pd.read_excel('Data_평가만료일_특허유지일.xlsx')
df = raw_df.iloc[:,2:-1]

In [18]:
X_train, X_test, y_train,y_test = train_test_split(df, raw_df.iloc[:,-1], test_size = 0.2, random_state=7)

# scaler = StandardScaler()
# scaler = MinMaxScaler()
scaler = RobustScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(188621, 27)
(47156, 27)
(188621,)
(47156,)


---

## 1. RF

In [19]:
def rf_objective(params):
    model = RandomForestRegressor(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        min_samples_split=int(params['min_samples_split']),
        min_samples_leaf=int(params['min_samples_leaf']),
        random_state=7,
        n_jobs=-1  # Enable parallel processing
    )
    
    # 5-Fold Cross Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=7)
    y_pred = cross_val_predict(model, X_train, y_train, cv=kf)
    
    MAPE, MAE, MSE, RMSE = get_regression_metrics(y_train, y_pred)
    
    return {
            'loss': MAE,  # Use MAE as the loss for optimization
            'status': STATUS_OK,  # Indicates that the evaluation was successful
            'mape': MAPE,
            'mae': MAE,
            'mse': MSE,
            'rmse': RMSE
        }


# Hyperparameter Space for RandomForest
rf_param_space = {
    'n_estimators': scope.int(hp.quniform('rf_n_estimators', 50, 300, 10)),
    'max_depth': scope.int(hp.quniform('rf_max_depth', 5, 30, 1)),
    'min_samples_split': scope.int(hp.quniform('rf_min_samples_split', 2, 10, 1)),
    'min_samples_leaf': scope.int(hp.quniform('rf_min_samples_leaf', 1, 5, 1)),
}

In [20]:
rf_trials = Trials()
best_rf = fmin(
    fn=rf_objective,
    space=rf_param_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=rf_trials
)

100%|██████████| 50/50 [1:19:22<00:00, 95.25s/trial, best loss: 868.71] 


In [21]:
print(f"Best RandomForest Parameters:{best_rf}")

Best RandomForest Parameters:{'rf_max_depth': 30.0, 'rf_min_samples_leaf': 1.0, 'rf_min_samples_split': 3.0, 'rf_n_estimators': 240.0}


## 2. XGBoost

In [22]:
# XGBoost GPU?
print(f"XGBoost Version: {xgb.__version__}")
try:
    dtrain = xgb.DMatrix(data=[[1, 2], [3, 4]], label=[0, 1])
    print("GPU support is enabled.")
except Exception as e:
    print("GPU support is not enabled:", e)

XGBoost Version: 2.1.1
GPU support is enabled.


In [23]:
def xgb_objective(params):
    model = xgb.XGBRegressor(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        tree_method='gpu_hist',  # for GPU Option
        random_state=7
    )
    
    # 5-Fold Cross Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=7)
    y_pred = cross_val_predict(model, X_train, y_train, cv=kf)
    
    MAPE, MAE, MSE, RMSE = get_regression_metrics(y_train, y_pred)
    
    return {
            'loss': MAE,  # Use MAE as the loss for optimization
            'status': STATUS_OK,  # Indicates that the evaluation was successful
            'mape': MAPE,
            'mae': MAE,
            'mse': MSE,
            'rmse': RMSE
    }


# Hyperparameter Space for RandomForest
xgb_param_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 300, 10),
    'max_depth': hp.quniform('max_depth', 3, 15, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

In [24]:
trials = Trials()
best_xgb_params = fmin(
    fn=xgb_objective,
    space=xgb_param_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

c:\Users\psh99\anaconda3\envs\IEEE_Access\lib\site-packages\xgboost\core.py:158: UserWarning: [19:15:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

c:\Users\psh99\anaconda3\envs\IEEE_Access\lib\site-packages\xgboost\core.py:158: UserWarning: [19:15:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:58: Falling back to prediction using DMatrix due to mismatched devices. This might lead to higher memory usage and slower performance. XGBoost is running on: cuda:0, while the input data is on: cpu.
Potential solutions:
- Use a data structure that matches the device ordinal in the boos

  2%|▏         | 1/50 [00:02<01:56,  2.37s/trial, best loss: 901.17]

c:\Users\psh99\anaconda3\envs\IEEE_Access\lib\site-packages\xgboost\core.py:158: UserWarning: [19:15:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

c:\Users\psh99\anaconda3\envs\IEEE_Access\lib\site-packages\xgboost\core.py:158: UserWarning: [19:15:44] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

c:\Users\psh99\anaconda3\envs\IEEE_Access\lib\site-packages\xgboost\core.py:15

  4%|▍         | 2/50 [00:04<01:39,  2.06s/trial, best loss: 901.17]

...

c:\Users\psh99\anaconda3\envs\IEEE_Access\lib\site-packages\xgboost\core.py:158: UserWarning: [19:44:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

c:\Users\psh99\anaconda3\envs\IEEE_Access\lib\site-packages\xgboost\core.py:158: UserWarning: [19:44:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

c:\Users\psh99\anaconda3\envs\IEEE_Access\lib\site-packages\xgboost\core.py:15

 98%|█████████▊| 49/50 [29:06<00:26, 26.24s/trial, best loss: 869.71]

c:\Users\psh99\anaconda3\envs\IEEE_Access\lib\site-packages\xgboost\core.py:158: UserWarning: [19:44:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

c:\Users\psh99\anaconda3\envs\IEEE_Access\lib\site-packages\xgboost\core.py:158: UserWarning: [19:44:47] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)

c:\Users\psh99\anaconda3\envs\IEEE_Access\lib\site-packages\xgboost\core.py:15

100%|██████████| 50/50 [29:21<00:00, 35.23s/trial, best loss: 869.71]


c:\Users\psh99\anaconda3\envs\IEEE_Access\lib\site-packages\xgboost\core.py:158: UserWarning: [19:45:02] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)



In [25]:
print("Best parameters:", best_xgb_params)

Best parameters: {'colsample_bytree': 0.852695929386359, 'learning_rate': 0.040387161559300055, 'max_depth': 15.0, 'n_estimators': 170.0, 'subsample': 0.8595758674808016}


## 3. LightGBM

In [26]:
def lgbm_objective(params):
    model = LGBMRegressor(
        n_estimators=int(params['n_estimators']),
        max_depth=int(params['max_depth']),
        learning_rate=params['learning_rate'],
        subsample=params['subsample'],
        colsample_bytree=params['colsample_bytree'],
        device='gpu',  # for GPU Option
        random_state=7
    )
    
    # 5-Fold Cross Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=7)
    y_pred = cross_val_predict(model, X_train, y_train, cv=kf)
    
    MAPE, MAE, MSE, RMSE = get_regression_metrics(y_train, y_pred)
    
    return {
            'loss': MAE,  # Use MAE as the loss for optimization
            'status': STATUS_OK,  # Indicates that the evaluation was successful
            'mape': MAPE,
            'mae': MAE,
            'mse': MSE,
            'rmse': RMSE
    }


# Hyperparameter Space for LightGBM
lgbm_param_space = {
    'n_estimators': hp.quniform('n_estimators', 50, 300, 10),
    'max_depth': hp.quniform('max_depth', 3, 15, 1),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.3),
    'subsample': hp.uniform('subsample', 0.5, 1.0),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.5, 1.0)
}

In [27]:
trials = Trials()
best_lgbm_params = fmin(
    fn=lgbm_objective,
    space=lgbm_param_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

[LightGBM] [Info] This is the GPU trainer!!           
[LightGBM] [Info] Total Bins 3399                     
[LightGBM] [Info] Number of data points in the train set: 150896, number of used features: 27
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] GPU programs have been built        
[LightGBM] [Info] Size of histogram bin entry: 8      
[LightGBM] [Info] 17 dense feature groups (2.88 MB) transferred to GPU in 0.002763 secs. 1 sparse feature groups
[LightGBM] [Info] Start training from score 2809.513292
[LightGBM] [Info] This is the GPU trainer!!           
[LightGBM] [Info] Total Bins 3395                     
[LightGBM] [Info] Number of data points in the train set: 150897, number of used features: 27
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce RTX 3060, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 256 bins...
[LightGBM] [Info] G

In [28]:
print("Best parameters:", best_lgbm_params)

Best parameters: {'colsample_bytree': 0.9976300732323532, 'learning_rate': 0.2176429779300096, 'max_depth': 10.0, 'n_estimators': 290.0, 'subsample': 0.5658544868665238}


## 4. LR

In [29]:
def evaluate_lr_model():
    model = LinearRegression()

    # 5-Fold Cross Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=7)
    y_pred = cross_val_predict(model, X_train, y_train, cv=kf)
    
    MAPE, MAE, MSE, RMSE = get_regression_metrics(y_train, y_pred)
    
    return {
            'loss': MAE,  # Use MAE as the loss for optimization
            'status': STATUS_OK,  # Indicates that the evaluation was successful
            'mape': MAPE,
            'mae': MAE,
            'mse': MSE,
            'rmse': RMSE
        }

In [30]:
lr_metrics = evaluate_lr_model()
print(f"regression metrics: {lr_metrics}")

regression metrics: {'loss': 957.3, 'status': 'ok', 'mape': 0.41, 'mae': 957.3, 'mse': 1355250.65, 'rmse': 1164.15}


## 5. SVR

In [31]:
def objective(params):
    model = make_pipeline(
        SVR(
            C=params['C'],
            epsilon=params['epsilon'],
            kernel=params['kernel']
        )
    )
    
    # 5-Fold Cross Validation
    kf = KFold(n_splits=5, shuffle=True, random_state=7)
    y_pred = cross_val_predict(model, X_train, y_train, cv=kf)
    
    MAPE, MAE, MSE, RMSE = get_regression_metrics(y_train, y_pred)

    # Return the loss (mean MAE for optimization) and other metrics
    return {
        'loss': MAE,  # Use MAE as the loss for optimization
        'status': STATUS_OK,  # Indicates that the evaluation was successful
        'mape': MAPE,
        'mae': MAE,
        'mse': MSE,
        'rmse': RMSE
    }
    
# Hyperparameter Space for SVR
param_space = {
    'C': hp.loguniform('C', -3, 3),  # 0.05부터 약 20까지
    'epsilon': hp.uniform('epsilon', 0.01, 1.0),
    'kernel': hp.choice('kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
}

In [ ]:
trials = Trials()
best_svr_params = fmin(
    fn=objective,
    space=param_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=trials
)

 90%|█████████ | 45/50 [65:18:55<13:00:09, 9361.98s/trial, best loss: 949.21] 

In [ ]:
print("Best parameters:", best_svr_params)

## 6. DNN

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Dense(units=hp.Int('units_1', min_value=64, max_value=256, step=32),
                    input_dim=X_train.shape[1], activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_1', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(units=hp.Int('units_2', min_value=32, max_value=128, step=16), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_2', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(units=hp.Int('units_3', min_value=16, max_value=64, step=8), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_3', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(units=hp.Int('units_4', min_value=8, max_value=32, step=4), activation='relu'))
    model.add(Dropout(rate=hp.Float('dropout_4', min_value=0.1, max_value=0.5, step=0.1)))

    model.add(Dense(1, activation='relu'))

    model.compile(
        optimizer=tf.keras.optimizers.Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='log')
        ),
        loss='mae',
        metrics=['mae']
    )
    return model


dnn_tuner = Hyperband(
    build_model,
    objective='val_mae',
    max_epochs=200,
    factor=3,
    directory='tuner_results',
    project_name='DNN_hyperparameter_tuning'
)

# K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=7)

# Define Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
checkpoint = ModelCheckpoint("DNN_model_best.h5", monitor='val_loss', save_best_only=True, mode='min')


Reloading Tuner from tuner_results\DNN_hyperparameter_tuning\tuner0.json


In [ ]:
type(X_train)


numpy.ndarray

In [ ]:
# Hyperparameter tuning through K-Fold Cross Validation
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    dnn_tuner.search(
        X_train_fold, y_train_fold,
        epochs=50,
        validation_data=(X_val_fold, y_val_fold),
        callbacks=[early_stopping, checkpoint]
    )

# Model Learning
best_dnn_params = dnn_tuner.get_best_hyperparameters(num_trials=1)[0]
dnn_model = dnn_tuner.hypermodel.build(best_dnn_params)
history = dnn_model.fit(
    X_train, y_train,
    epochs=200,
    batch_size=128,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping, checkpoint]
)


In [ ]:
print(best_dnn_params.values)

## 7. AE

In [ ]:
def build_autoencoder(hp):
    encoding_dim = hp.Choice('encoding_dim', [16, 32, 64, 128])  # 튜닝 가능한 encoding_dim
    input_layer = Input(shape=(X_train.shape[1],))  # 입력 크기 설정
    encoded = Dense(encoding_dim, activation='relu')(input_layer)  # 인코더 레이어
    decoded = Dense(X_train.shape[1], activation=hp.Choice('decoder_activation', ['sigmoid', 'relu']))(encoded)  # 디코더 레이어
    autoencoder = Model(input_layer, decoded)
    autoencoder.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),  # 튜닝 가능한 학습률
        loss=MeanAbsoluteError()
    )
    return autoencoder


def build_regressor(hp):
    model = Sequential()
    model.add(Dense(
        hp.Int('units_1', min_value=32, max_value=128, step=32),
        activation='relu',
        input_dim=fixed_encoding_dim
    ))
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(Dense(
            hp.Int(f'units_{i+2}', min_value=16, max_value=64, step=16),
            activation='relu'
        ))
    model.add(Dense(1, activation='linear'))
    model.compile(
        optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss=MeanAbsoluteError(),
        metrics=['mae']
    )
    return model


early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Set K-Fold Cross Validation
kf = KFold(n_splits=5, shuffle=True, random_state=7)

In [ ]:
autoencoder_tuner = kt.Hyperband(
    build_autoencoder,
    objective='val_loss',
    max_epochs=50,
    factor=3,
    directory='my_dir',
    project_name='autoencoder_tuning'
)


In [ ]:
# Autoencoder Tuning
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]

    autoencoder_tuner.search(X_train_fold, X_train_fold, validation_data=(X_val_fold, X_val_fold), epochs=50, batch_size=256)

# get Autoencoder parameter
best_hp_autoencoder = autoencoder_tuner.get_best_hyperparameters(1)[0]
autoencoder = autoencoder_tuner.hypermodel.build(best_hp_autoencoder)

encoder = Model(autoencoder.input, autoencoder.layers[1].output)

encoded_X_train = encoder.predict(X_train)
encoded_X_test = encoder.predict(X_test)

fixed_encoding_dim = best_hp_autoencoder.get('encoding_dim')

# Regressor tuning
regressor_tuner = kt.Hyperband(
    build_regressor,
    objective='val_mae',
    max_epochs=50,
    factor=3,
    directory='my_dir',
    project_name='regressor_tuning'
)

In [ ]:
# K-Fold
for train_index, val_index in kf.split(encoded_X_train):
    X_train_fold, X_val_fold = encoded_X_train[train_index], encoded_X_train[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]
    
    regressor_tuner.search(X_train_fold, y_train_fold, validation_data=(X_val_fold, y_val_fold), epochs=50, batch_size=32)

# apply hyperparameter 
best_hp_regressor = regressor_tuner.get_best_hyperparameters(1)[0]
regressor = regressor_tuner.hypermodel.build(best_hp_regressor)

# Final Learning
regressor.fit(encoded_X_train, y_train, epochs=100, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

# Prediction
auto_encoder_predictions = regressor.predict(encoded_X_test)
print_test_results(y_test, auto_encoder_predictions)